In [7]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Optional, Sequence

import numpy as np
import pandas as pd

FOUR_PLAYER_RECORD_SHEET = "点数表_四麻"


In [8]:

@dataclass(frozen=True)
class PlayerSegment:
    """Logical grouping of score, cumulative, and rank columns for a player."""

    name: str
    score: str
    cumulative: Optional[str]
    rank: Optional[str]

In [9]:
def _load_raw_four_player_sheet(path: Path) -> tuple[pd.DataFrame, str, list[PlayerSegment]]:
    """Load the four-player record sheet and detect per-player column segments."""
    df = pd.read_excel(path, sheet_name=FOUR_PLAYER_RECORD_SHEET, header=0)
    df = df.dropna(how="all")

    columns = df.columns.tolist()
    if not columns:
        raise ValueError("点数表_四麻 シートにヘッダーが見つかりませんでした。")

    date_column = columns[0]
    segments: List[PlayerSegment] = []

    def _contains_any(value: object, keywords: Iterable[str]) -> bool:
        if not isinstance(value, str):
            return False
        return any(keyword in value for keyword in keywords)

    cumulative_keywords = ("累", "合計", "total", "TOTAL")
    rank_keywords = ("順位", "着順", "rank", "RANK")

    i = 1
    while i < len(columns):
        name = columns[i]
        if not isinstance(name, str):
            i += 1
            continue
        if not name.strip() or name.startswith("Unnamed"):
            i += 1
            continue
        if _contains_any(name, cumulative_keywords) or _contains_any(name, rank_keywords):
            i += 1
            continue

        cumulative: Optional[str] = None
        rank: Optional[str] = None
        next_index = i + 1

        if next_index < len(columns) and _contains_any(columns[next_index], cumulative_keywords):
            cumulative = columns[next_index]
            next_index += 1

        if next_index < len(columns) and _contains_any(columns[next_index], rank_keywords):
            rank = columns[next_index]
            next_index += 1

        segments.append(PlayerSegment(name=name, score=name, cumulative=cumulative, rank=rank))
        i = next_index

    if not segments:
        raise ValueError("プレイヤー列を特定できませんでした。シート構成を確認してください。")

    return df, date_column, segments

In [10]:
path = Path.cwd().parent/'data'/'理物麻雀_2025_26.xlsx'
_load_raw_four_player_sheet(path)

(          日付     きよ     合計点数   順位     やまだ  合計点数.1  順位.1  よしたに  合計点数.2  順位.2  \
 0    45931.0 -18.30   -18.30  3.0    5.40    5.40   2.0   0.0     0.0   0.0   
 1        NaN -27.05   -45.35  3.0  -71.55  -66.15   4.0   0.0     0.0   0.0   
 2        NaN -21.60   -66.95  3.0  -62.50 -128.65   4.0   0.0     0.0   0.0   
 3        NaN -75.60  -142.55  4.0   22.00 -106.65   2.0   0.0     0.0   0.0   
 4        NaN -47.00  -189.55  3.0  126.90   20.25   1.0   0.0     0.0   0.0   
 ..       ...    ...      ...  ...     ...     ...   ...   ...     ...   ...   
 994      NaN   0.00  1069.40  NaN    0.00 -181.20   NaN   0.0  -353.6   NaN   
 995      NaN   0.00  1069.40  NaN    0.00 -181.20   NaN   0.0  -353.6   NaN   
 996      NaN   0.00  1069.40  NaN    0.00 -181.20   NaN   0.0  -353.6   NaN   
 997      NaN   0.00  1069.40  NaN    0.00 -181.20   NaN   0.0  -353.6   NaN   
 998      NaN   0.00  1069.40  NaN    0.00 -181.20   NaN   0.0  -353.6   NaN   
 
      ...  合計点数.3  順位.3     れい  合計点数.4

In [11]:
pd.read_excel(path,FOUR_PLAYER_RECORD_SHEET)

,日付,きよ,合計点数,順位,やまだ,合計点数.1,順位.1,よしたに,合計点数.2,順位.2,...,合計点数.3,順位.3,れい,合計点数.4,順位.4,ひなた,合計点数.5,順位.5,全合計,Unnamed: 20
0,45931.0,-18.30,-18.30,3.0,5.40,5.40,2.0,0.0,0.0,0.0,...,59.00,1.0,-46.10,-46.10,4.0,0.0,0.0,0,0,NaN
1,NaN,-27.05,-45.35,3.0,-71.55,-66.15,4.0,0.0,0.0,0.0,...,75.65,2.0,81.95,35.85,1.0,0.0,0.0,0,0,NaN
2,NaN,-21.60,-66.95,3.0,-62.50,-128.65,4.0,0.0,0.0,0.0,...,145.75,1.0,14.00,49.85,2.0,0.0,0.0,0,0,NaN
3,NaN,-75.60,-142.55,4.0,22.00,-106.65,2.0,0.0,0.0,0.0,...,227.65,1.0,-28.30,21.55,2.0,0.0,0.0,0,0,NaN
4,NaN,-47.00,-189.55,3.0,126.90,20.25,1.0,0.0,0.0,0.0,...,158.75,4.0,-11.00,10.55,2.0,0.0,0.0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,0.00,1069.40,NaN,0.00,-181.20,NaN,0.0,-353.6,NaN,...,-192.40,NaN,0.00,-274.10,NaN,0.0,-73.1,0,0,NaN
995,NaN,0.00,1069.40,NaN,0.00,-181.20,NaN,0.0,-353.6,NaN,...,-192.40,NaN,0.00,-274.10,NaN,0.0,-73.1,0,0,NaN
996,NaN,0.00,1069.40,NaN,0.00,-181.20,NaN,0.0,-353.6,NaN,...,-192.40,NaN,0.00,-274.10,NaN,0.0,-73.1,0,0,NaN
997,NaN,0.00,1069.40,NaN,0.00,-181.20,NaN,0.0,-353.6,NaN,...,-192.40,NaN,0.00,-274.10,NaN,0.0,-73.1,0,0,NaN
